In [13]:
import pdp_instance_generator as ig
import numpy as np

seed = 1241;
grid_km = 10;
num_requests = 20;
data = ig.generate_basic_instance(seed, grid_km, num_requests)

# data keys (generate_basic_instance input)
# grid_km = side of the area 
# num_requests = number of requests
# seed = random seed for generating instances

# data keys (generate_basic_instance output)
# nodes = list of nodes, each element has id and x,y coordinates
# distance_matrix = all distances between each couple of nodes
# depot = id 0 and position in the center of the area 
# pickups_deliveries = list of requests, each element has node ids for pickup and delivery 

# data keys (manually set)
# num_vehicles = number of vehicles (all assigned to the same depot)
# max_travel_distance_vehicle = vehicle route distance can be at most this value, 9999999 very large distance (basically no constraint) 
data['num_vehicles'] = 3
data['max_travel_distance_vehicle'] = 9999999

# demand/capacity data keys (manually set)
# Both mandatory to activate capacity constraints
# vehicle_capacities = list of capacity for each vehicle 
# demands = simply demand 1 and -1 to pickup and delivery nodes
# commentare i parametri sotto per non far mettere i vincoli di capacità nel modello
data['vehicle_capacities'] = [1,20,1]
ig.add_demands(data)

#aggiungi penalità
#ig.add_penalties(data, 3000)

print(data.keys())
print(data)

dict_keys(['nodes', 'distance_matrix', 'depot', 'pickups_deliveries', 'grid_km', 'num_requests', 'seed', 'num_vehicles', 'max_travel_distance_vehicle', 'vehicle_capacities', 'demands'])
{'nodes': [{'id': 0, 'x': 5000.0, 'y': 5000.0}, {'id': 1, 'x': 6017.9, 'y': 4959.39}, {'id': 2, 'x': 2115.48, 'y': 2530.86}, {'id': 3, 'x': 1951.66, 'y': 4908.4}, {'id': 4, 'x': 7368.24, 'y': 5265.1}, {'id': 5, 'x': 2835.36, 'y': 525.89}, {'id': 6, 'x': 4771.07, 'y': 1951.23}, {'id': 7, 'x': 8991.05, 'y': 3852.4}, {'id': 8, 'x': 2678.37, 'y': 9541.26}, {'id': 9, 'x': 6783.31, 'y': 424.53}, {'id': 10, 'x': 1483.23, 'y': 749.96}, {'id': 11, 'x': 4931.74, 'y': 5113.39}, {'id': 12, 'x': 2597.03, 'y': 7386.9}, {'id': 13, 'x': 3172.63, 'y': 43.81}, {'id': 14, 'x': 5768.34, 'y': 6756.52}, {'id': 15, 'x': 8009.55, 'y': 4168.95}, {'id': 16, 'x': 1707.64, 'y': 3544.28}, {'id': 17, 'x': 4401.26, 'y': 4570.09}, {'id': 18, 'x': 4981.81, 'y': 7091.09}, {'id': 19, 'x': 652.32, 'y': 5398.17}, {'id': 20, 'x': 1198.34, '

In [14]:
import pdp_or_tools as ort
# Capacity constraints are skipped if data don't have'demands' and 'vehicle_capacities' fields
manager, routing, solution = ort.solve(data)

Added demand and capacity constraints
Skipping unsatisfied demand penalties: data has no 'penalties' field
It's mandatory to satisfy all demand
Instance loaded
Solving time: 0.38 seconds
Model solved.


In [15]:
import pdp_plot as plt

plt.plot_solution(manager, routing, solution, data)